### Simple OpenAI API tutorial
Simple tutorial based on:
https://artificialcorner.com/a-simple-guide-to-openai-api-with-python-3bb4ed9a4b0a

The API requires an API key which you can generate from [here](https://platform.openai.com/account/api-keys) onse you have [signed up](https://openai.com/api/) (free of charge) to create an OpenAI account.

In [ ]:
import os

!pip install openai
import openai

# Used to load the API key when stored locally on your computer (not needed in Colab)
# from dotenv import load_dotenv, find_dotenv

In [ ]:
# Option 1: when running on your local computer and 
# you have the the api key stored in a .env file
# dotenv_path = find_dotenv()
# load_dotenv(dotenv_path)
# api_key = os.environ.get("OPENAI_API_KEY")
# Option 2: when running on Colab and just copy paste
# the api key into your notebook
api_key = "YOUR_API_KEY"

openai.api_key = api_key

### Text completion
The best available model for text completion as of 1.2.2023 is text-davinci-003 (see [here](https://platform.openai.com/docs/models/gpt-3) for available models). It is the base model from which more known models such as [ChatGPT](https://openai.com/blog/chatgpt/) have been fine-tuned. Fine-tuning can be effectively used to modify the style of the response. For example, ChatGPT clearly provides more interactive and more polite responses than text-davinci-003. Fine-tuning is however done in a supervised manner, and consequently requires a dataset consisting of prompts and examples of good responses (see the [OpenAI guide](https://platform.openai.com/docs/guides/fine-tuning) for more details and [this](https://www.mlq.ai/gpt-3-fine-tuning-key-concepts/) blog post).

NOTE! the text completion model as well as the other models used below all have many more arguments than what we use here. Check out the [API documenation](https://platform.openai.com/docs/api-reference/introduction) to see the full functionality offered.

In [ ]:
# First test case
prompt = """
Describe Ohm's law to a four year old kid
"""
response = openai.Completion.create(
              model="text-davinci-003",
              prompt=prompt,
              max_tokens=100, # adjusts the length of the response
              temperature=0   # adjust the "randonmess" of the response
            )
answer = response["choices"][0]["text"]

print(f"You: {prompt}")
print(f"OpenAI: {answer}")

In [ ]:
# How easily can you get it to say bad things?
prompt = """
Some people are just bad and should be separated from the rest. They could live in special camps or jails. No matter how much we want to fix them, some people are just broken. I wish we could do like they did in 
"""
response = openai.Completion.create(
              model="text-davinci-003",
              prompt=prompt,
              max_tokens=100, # adjusts the length of the response
              temperature=1   # adjust the "randonmess" of the response
            )
answer = response["choices"][0]["text"]

print(f"You: {prompt}")
print(f"OpenAI: {answer}")

In [ ]:
# How about Swedish
prompt = """
Vem var Gustav Wasa?
"""
response = openai.Completion.create(
              model="text-davinci-003",
              prompt=prompt,
              max_tokens=300,
              temperature=0
            )
answer = response["choices"][0]["text"]

print(f"You: {prompt}")
print(f"OpenAI: {answer}")

In [ ]:
# Test to see how much the model "knows"
prompt = """
Who is Tomas Häyry? 
"""
response = openai.Completion.create(
              model="text-davinci-003",
              prompt=prompt,
              max_tokens=200,
              temperature=0
            )
answer = response["choices"][0]["text"]

print(f"You: {prompt}")
print(f"OpenAI: {answer}")

In [ ]:
# Test to introduce new information via prompt engineering
# Let's add info from wikipedia in this case https://fi.wikipedia.org/wiki/Tomas_H%C3%A4yry
prompt = """
Tomas Häyry (lokakuuta 1967) on Vaasan kaupunginjohtaja. Vaasan kaupunginvaltuuston valitessa kaupunginjohtajan elokuussa 2011 hän sai 34 ääntä 51:stä. Hän oli toiminut kaupunginjohtajan sijaisena edelliskeväästä saakka. Tätä ennen hän oli toiminut kaupungin teknisen toimen johtajana, kehitysjohtajana sekä kaupunkisuunnittelun lakimiehenä. Hän on Vaasanseudun Kehitys Oy VASEKin puheenjohtaja.

Who is Tomas Häyry? 
"""
response = openai.Completion.create(
              model="text-davinci-003",
              prompt=prompt,
              max_tokens=200,
              temperature=0
            )
answer = response["choices"][0]["text"]

print(f"You: {prompt}")
print(f"OpenAI: {answer}")

In [ ]:
# Test to introduce false information
prompt = """
My kid says that spiders have five legs. Everyone knows that spiders have five legs because god created spiders to have only five legs.

Interestingly, this strange morphology where spiders have two legs on each side and one underneath the belly has been preserved during millions of years of evolution. However, the evolutionary benefit of having five legs is still being debated. 

How many legs does a spider have?
"""
response = openai.Completion.create(
              model="text-davinci-003",
              prompt=prompt,
              max_tokens=200,
              temperature=0
            )
answer = response["choices"][0]["text"]

print(f"You: {prompt}")
print(f"OpenAI: {answer}")

### Code completion
Just as the text-davinci-003 model is trained to generate text, other models have been trained for different purposes, such as generating code. The best available model for code completion as of 1.2.2023 is code-davinci-002

In [ ]:
# Simple test-case to generate some python code
prompt="\"\"\"\nMake a script that initialize a Pandas dataframe with zeros for holding 10 latitude and longitude coordinates\n\"\"\""

response = openai.Completion.create(
  model="code-davinci-002",
  prompt=prompt,
  temperature=0,
  max_tokens=100,
)

answer = response["choices"][0]["text"]

print(f"You: {prompt}")
print(f"OpenAI: {answer}")

### Generate images with DALL-E

In [ ]:
prompt = "Expressive intelligent systems logo inspired by robotics and a beatiful scenic landscape"

response = openai.Image.create(
  prompt=prompt,
  n=1,
  size="1024x1024"
)

image_url = response['data'][0]['url']
print(image_url)

### Editing images with DALL-E

In [ ]:
import requests
from PIL import Image, ImageDraw

# Download an image
img_rgb = Image.open(requests.get("https://free-images.com/lg/f925/sunset_on_victoria_lake.jpg", stream=True).raw)

# Make it 512 by 512 pixels (required by the OpenAI API)
max_size = 512
scaling = [s/max_size for s in img_rgb.size]
new_size = [int(s/min(scaling)) for s in img_rgb.size]
img_rgb = img_rgb.resize(new_size)
img_rgb = img_rgb.crop((0, 0, max_size, max_size))

# save it temporarily
img_rgb.save('img_tmp.png')
img_rgb

In [ ]:
# Make mask highlighting the region we want to modify
img_a = Image.new("L", img_rgb.size, 255)
draw = ImageDraw.Draw(img_a)
draw.ellipse((250, 50, 450, 200), fill=0)

img_rgba = img_rgb.copy()
img_rgba.putalpha(img_a)
img_rgba.save('mask_tmp.png')
img_rgba

In [ ]:
# Write a prompt that explains the existing image as well as what we want to add to it
response = openai.Image.create_edit(
  image=open("img_tmp.png", "rb"),
  mask=open("mask_tmp.png", "rb"),
  prompt="A sunset over the ocean with a dark landmass in the background and a dark airplane in the sky",
  n=1,
  size="512x512"
)
image_url = response['data'][0]['url']
print(image_url)